In [84]:
import requests
from dotenv import load_dotenv
import os
import json
import pandas as pd
from datetime import datetime


In [85]:
load_dotenv("env")
NOTION_SECRET = os.getenv('notion_secret')
CAL_ID = os.getenv('calendario_id')
LDC_ID = os.getenv('lista_de_compra_id')

In [105]:
def read_data(api_secret:str, db_id:str, dump_json:bool = False, num_pages:int=None):
    headers = {
    "Authorization" : "Bearer " + api_secret,
    "Content-Type" : "application/json",
    "Notion-Version" : "2022-06-28"
    }
    endpoint_url =  f'https://api.notion.com/v1/databases/{db_id}/query'
    get_all = num_pages is None
    page_size = 100 if get_all else num_pages
    payload = {"page_size" : page_size}

    response = requests.post(endpoint_url, json = payload, headers=headers)

    if response.status_code == 404:
        print("Ha habido un error 404. La base de datos no existe")
        
    data = response.json()
    
    try:
        results = data['results']
        
        while data['has_more'] and get_all:
            payload = {
                'page_size' : page_size,
                'start_cursor' : data['next_cursor']
            }
            response = requests.post(endpoint_url, json = payload, headers=headers)
            data = response.json()
            results.extend(data['results'])

        if dump_json:
            with open('dump.json', 'w') as f:
                json.dump(results, f, indent = 4)

        return results
    
    except:
        print("Ha habido un error en la conexión")
        print(data)
        return data        



In [134]:
def create_page(api_secret:str, db_id: str, data:dict):
    create_endpoint = "https://api.notion.com/v1/pages"
    headers = {
    "Authorization" : "Bearer " + api_secret,
    "Content-Type" : "application/json",
    "Notion-Version" : "2022-06-28"
    }
    payload = {
        "parent" : {
            "database_id" : db_id
        },
        "properties" : data
    }

    result = requests.post(create_endpoint, headers = headers, json = payload)
    print(result.status_code)
    return result

In [136]:
def listacompradata(ingrediente:str, check:str = "false"):
    dct = {
        "ingrediente" : ingrediente,
        "check" : check
    }
    page = """
        {
            "Comprado": {
                "id": "rGpQ",
                "type": "checkbox",
                "checkbox": %(check)s
            },
            "Ingrediente": {
                "id": "title",
                "type": "title",
                "title": [
                    {
                        "type": "text",
                        "text": {
                            "content": "%(ingrediente)s",
                            "link": null
                        },
                        "annotations": {
                            "bold": false,
                            "italic": false,
                            "strikethrough": false,
                            "underline": false,
                            "code": false,
                            "color": "default"
                        },
                        "plain_text": "%(ingrediente)s",
                        "href": null
                    }
                ]
            }
        }
    """ % dct
    
    return json.loads(page)

In [107]:

test = read_data(NOTION_SECRET, LDC_ID, dump_json=True)

In [133]:
listacompradata("hola")

{'Etiquetas': {'id': 'rGpQ', 'type': 'checkbox', 'checkbox': False},
 'Ingrediente': {'id': 'title',
  'type': 'title',
  'title': [{'type': 'text',
    'text': {'content': 'hola', 'link': None},
    'annotations': {'bold': False,
     'italic': False,
     'strikethrough': False,
     'underline': False,
     'code': False,
     'color': 'default'},
    'plain_text': 'hola',
    'href': None}]}}

In [138]:
create_page(
    api_secret=NOTION_SECRET,
    db_id=LDC_ID,
    data = listacompradata("test")
)

200


<Response [200]>

In [98]:
LDC_ID

'a6f4c95fd341444fa129790761c285ab'

In [101]:
test

{'object': 'list',
 'results': [{'object': 'page',
   'id': '0f90b67b-f21c-4c18-aa8f-81b189f7aa5b',
   'created_time': '2024-05-19T10:49:00.000Z',
   'last_edited_time': '2024-05-19T10:51:00.000Z',
   'created_by': {'object': 'user',
    'id': '8a9c61c8-1712-4558-8eac-f0c91382ee7a'},
   'last_edited_by': {'object': 'user',
    'id': '8a9c61c8-1712-4558-8eac-f0c91382ee7a'},
   'cover': None,
   'icon': None,
   'parent': {'type': 'database_id',
    'database_id': 'a6f4c95f-d341-444f-a129-790761c285ab'},
   'archived': False,
   'in_trash': False,
   'properties': {'Etiquetas': {'id': 'rGpQ',
     'type': 'checkbox',
     'checkbox': True},
    'Ingrediente': {'id': 'title',
     'type': 'title',
     'title': [{'type': 'text',
       'text': {'content': 'Macarrones', 'link': None},
       'annotations': {'bold': False,
        'italic': False,
        'strikethrough': False,
        'underline': False,
        'code': False,
        'color': 'default'},
       'plain_text': 'Macarrones'

In [90]:
query_endpoint = f'https://api.notion.com/v1/databases/{CAL_ID}/query'

test = read_data(NOTION_SECRET, query_endpoint)


In [91]:
test

[{'object': 'page',
  'id': '05260d04-8430-4034-bd94-39de6d0ef6eb',
  'created_time': '2024-05-14T18:52:00.000Z',
  'last_edited_time': '2024-05-14T18:55:00.000Z',
  'created_by': {'object': 'user',
   'id': 'ca0978e2-a385-4a0c-9107-2fccac7f1be0'},
  'last_edited_by': {'object': 'user',
   'id': 'ca0978e2-a385-4a0c-9107-2fccac7f1be0'},
  'cover': None,
  'icon': {'type': 'emoji', 'emoji': '🦐'},
  'parent': {'type': 'database_id',
   'database_id': '740d547f-c866-4bbb-bfd2-c1150e956b94'},
  'archived': False,
  'in_trash': False,
  'properties': {'Ingredientes': {'id': '%3ApWK',
    'type': 'multi_select',
    'multi_select': [{'id': '331f6934-d00e-4bee-9c42-24a32be027e4',
      'name': 'embutido',
      'color': 'brown'}]},
   'quien se la come': {'id': '%3D%5Biv',
    'type': 'people',
    'people': [{'object': 'user',
      'id': 'ca0978e2-a385-4a0c-9107-2fccac7f1be0',
      'name': 'Ague G.',
      'avatar_url': 'https://lh3.googleusercontent.com/a-/AOh14Gh8F23pE8JJmHAn8xFq6IneGUr-O

In [92]:
for page in test:
    if not page['properties']['Comprado']['checkbox']:
        for ing in page['properties']['Ingredientes']['multi_select']:
            print(ing['name']) # esto sería meterlo a una lista o algo

embutido


In [53]:
data['results'][2]['properties']['Ingredientes']['multi_select'][0]['name']


In [4]:
hmm = pd.DataFrame(data['results'])
hmmm = hmm['properties']

In [7]:
si = pd.DataFrame(pd.DataFrame(hmmm.to_list())['Ingredientes'].to_list())['multi_select'].to_list()

In [8]:
pd.DataFrame(si)

,0,1,2,3,4,5,6
0,"{'id': '331f6934-d00e-4bee-9c42-24a32be027e4',...",None,None,None,None,None,None
1,None,None,None,None,None,None,None
2,"{'id': '55b81bec-40be-45e1-a72f-5406b0189b44',...","{'id': '5c445d7d-c76e-444c-93b5-c4ac96830a7f',...","{'id': 'be860c91-b0e9-4e2e-8470-16f3e3f6ddf8',...","{'id': '331f6934-d00e-4bee-9c42-24a32be027e4',...","{'id': 'ce13d018-2108-4bd3-8be6-ccb31c675337',...","{'id': '1443e577-183d-4108-8fa9-2c640a9b9190',...","{'id': 'c820d4fb-e16b-421e-ba89-97ee15247ed6',..."
3,"{'id': '70cee870-9fe9-40d7-9f5e-3f6a0a09c85c',...","{'id': '13f0eef6-057f-4a6b-9eb3-a2a2d676de52',...","{'id': '1443e577-183d-4108-8fa9-2c640a9b9190',...","{'id': 'e8bbdbf2-e3df-4afb-bae3-b2b37d9e4a84',...",None,None,None
